In [1]:
using LowRankModels
using CSV
using DataFrames
using Pkg
using StatsBase
using Random, SparseArrays

In [2]:
df3 = CSV.read("/Users/kathy908000/github/health-care-research/data/Diabetes_Numerical.csv")
df3
#first 3 col numerical
#other col boolean

,SUBJECT_ID,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN
,Int64,Float64,Int64,Int64,Int64,Int64,Int64
1,13,3.666,1,4,0,1,0
2,18,1.2885,1,6,0,1,0
3,20,1.0508,1,8,1,0,0
4,21,14.2664,2,9,0,1,0
5,24,0.5124,1,4,0,1,0
6,25,3.5466,1,6,0,1,0
7,28,1.1224,1,8,1,0,0
8,35,5.3757,1,7,1,0,0
9,37,1.1397,1,7,0,1,0


In [3]:
#drop columns
#df3 = select!(df3, Not(:Column1))
df3 = select!(df3, Not(:SUBJECT_ID )) 
df3

,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government
,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3.666,1,4,0,1,0,0,0
2,1.2885,1,6,0,1,0,0,0
3,1.0508,1,8,1,0,0,0,0
4,14.2664,2,9,0,1,0,0,0
5,0.5124,1,4,0,1,0,0,0
6,3.5466,1,6,0,1,0,0,0
7,1.1224,1,8,1,0,0,0,0
8,5.3757,1,7,1,0,0,0,0
9,1.1397,1,7,0,1,0,0,0


In [20]:
#k=6

In [11]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [12]:
k = 6
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm16 = GLRM(A,losses,rx,ry,k)
X16,Y16,ch16 = fit!(glrm16)

Fitting GLRM
Iteration 10: objective value = 1.2358809590389237e6
Iteration 20: objective value = 467863.6837298325
Iteration 30: objective value = 460131.0032340185
Iteration 40: objective value = 458166.17883389874


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0], [3.0341833701471033 1.26773761141311 … -1.1523364782875667 -1.1304504458498537; 2.96400576529314 1.2663371979590115 … -1.2074235485901295 -1.1766318805564684; … ; 3.057614574974197 1.2328879585477708 … -1.072302064836926 -1.042230979797023; 3.2224287177686866 1.3234592433833166 … -1.0428207080928356 -1.0999365306313769], ConvergenceHistory("ProxGradGLRM", [Inf, 9.45741176845789e6, 8.188791081082007e6, 6.963971326735926e6, 5.801918714053964e6, 4.721280265183693e6, 3.7382619554868536e6, 2.8949541317370883e6, 2.203161101773608e6, 1.6554570768934921e6  …  459451.1557345492, 459246.0739777007, 459044.06078413624, 458883.7866358956, 458690.7826032172, 458549.52579626086, 458449.87060531304, 458325.4149884006, 458166.17883389874, 458079.99813387636], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [13]:
Y16

6×920 Array{Float64,2}:
 3.03418  1.26774  7.0001   -1.00011  …  -1.0959   -1.15234  -1.13045
 2.96401  1.26634  7.02226  -1.00017     -1.0      -1.20742  -1.17663
 2.95308  1.28353  6.99977  -1.00118     -1.16977  -1.05262  -1.10441
 2.95152  1.32192  7.00331  -1.00001     -1.21888  -1.00229  -1.19015
 3.05761  1.23289  7.00145  -1.00057     -1.12334  -1.0723   -1.04223
 3.22243  1.32346  7.00224  -1.00021  …  -2.3115   -1.04282  -1.09994

In [14]:
imputey6 = impute(losses, Y16) 
imputey6d  = convert(DataFrame, imputey6) 
CSV.write("/Users/kathy908000/github/health-care-research/data/iy6.csv",  imputey6d)
CSV.read("/Users/kathy908000/github/health-care-research/data/iy6.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
,Float64,Float64,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,3.03418,1.26774,7,0,1,0,0,0,0,1,0,0,0
2,2.96401,1.26634,7,0,1,0,0,0,0,1,0,0,0
3,2.95308,1.28353,7,0,1,0,0,0,0,1,0,0,0
4,2.95152,1.32192,7,0,1,0,0,0,0,1,0,0,0
5,3.05761,1.23289,7,0,1,0,0,0,0,1,0,0,0
6,3.22243,1.32346,7,0,1,0,0,0,0,1,0,0,0


In [24]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm16, nfolds=2, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 4741735 samples and testing on 4743465
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 228479.22564116726
computing train and test error for fold 1:
	train error: 0.048181994985229216
	test error:  0.047844071938552835

forming train and test GLRM for fold 2
training model on 4743465 samples and testing on 4741735
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 226939.5290506657
computing train and test error for fold 2:
	train error: 0.047839298777992895
	test error:  0.048187109942916286


,train_error,test_error
,Float64,Float64
1,0.048182,0.0478441
2,0.0478393,0.0481871


In [25]:
hingek6 = convert(DataFrame, X16) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [26]:
CSV.write("/Users/kathy908000/github/health-care-research/data/hingek6.csv",  hingek6)
CSV.read("/Users/kathy908000/github/health-care-research/data/hingek6.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [27]:
#k=5

In [28]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [29]:
k = 5
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm05 = GLRM(A,losses,rx,ry,k)
X05,Y05,ch05 = fit!(glrm05)

Fitting GLRM
Iteration 10: objective value = 785742.1835691773
Iteration 20: objective value = 464382.6973364457
Iteration 30: objective value = 458976.40695412154


([0.0 0.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [3.0438699090695214 1.2190745202970292 … -1.1476688477673724 -1.1875806068055965; 3.070655940227776 1.2805376728741802 … -1.1112362103123397 -1.0725763698211597; … ; 3.085735626224026 1.2959990948628408 … -1.0221085405728578 -1.003942657973069; 2.8556297748416637 1.267715142929081 … -1.635254380404446 -1.1576591530418048], ConvergenceHistory("ProxGradGLRM", [Inf, 9.229352802910212e6, 7.715683671581579e6, 6.285400685113965e6, 4.980678349513673e6, 3.8275609943980626e6, 2.844701641404502e6, 2.0547237964759464e6, 1.4659329203738922e6, 1.0502594019564886e6  …  461119.6792584686, 460364.1836527265, 459997.1796645072, 459568.4350020009, 459315.99595636333, 459141.2873410248, 458976.40695412154, 458793.5628206643, 458641.82994953723, 458556.1529098767], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [30]:
X05

5×10310 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0  1.0
 0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0

In [31]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm05, nfolds=2, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 4744508 samples and testing on 4740692
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 229062.5706120421
computing train and test error for fold 1:
	train error: 0.048277291933473014
	test error:  0.04829102720400767

forming train and test GLRM for fold 2
training model on 4740692 samples and testing on 4744508
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 228924.74098281117
computing train and test error for fold 2:
	train error: 0.04828692790486687
	test error:  0.048286842386987175


,train_error,test_error
,Float64,Float64
1,0.0482773,0.048291
2,0.0482869,0.0482868


In [32]:
hingek5 = convert(DataFrame, X05) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [33]:
CSV.write("/Users/kathy908000/github/health-care-research/data/hingek5.csv",  hingek5)
CSV.read("/Users/kathy908000/github/health-care-research/data/hingek5.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [34]:
#k=4

In [35]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [36]:
k = 4
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm04 = GLRM(A,losses,rx,ry,k)
X04,Y04,ch04 = fit!(glrm04)

Fitting GLRM
Iteration 10: objective value = 547044.3289933192
Iteration 20: objective value = 461629.06711909064
Iteration 30: objective value = 458055.4999498729


([0.0 1.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0; 1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [3.0236190926115536 1.2829358647705784 … -1.043216123036991 -1.2278324505200806; 3.054939247945164 1.2396055453378858 … -1.1214212177473875 -1.064811375785501; 2.9587691312733218 1.2444846227643045 … -1.1055352091988093 -1.661344278156245; 3.0301126552384474 1.2554976849661785 … -1.1061066642349748 -1.2708967200992287], ConvergenceHistory("ProxGradGLRM", [Inf, 8.809639344280688e6, 7.009860164855971e6, 5.369642737167674e6, 3.9478870062831566e6, 2.7911244754224205e6, 1.9063565008885416e6, 1.2830803117652684e6, 883661.1726983065, 658002.8247476513  …  459224.2372739171, 458932.7565060489, 458730.8953080178, 458543.30483791104, 458348.26080058375, 458184.4547892782, 458055.4999498729, 457941.25347550143, 457754.05841062596, 457689.0568375702], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0

In [37]:
X04

4×10310 Array{Float64,2}:
 0.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  0.0  1.0  0.0     1.0  0.0  0.0  0.0  0.0  1.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  1.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0

In [38]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm04, nfolds=2, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 4743656 samples and testing on 4741544
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 228460.42915155916
computing train and test error for fold 1:
	train error: 0.04815914775906654
	test error:  0.0482536951830532

forming train and test GLRM for fold 2
training model on 4741544 samples and testing on 4743656
fitting train GLRM for fold 2
Fitting GLRM
Iteration 10: objective value = 228764.8008313564
computing train and test error for fold 2:
	train error: 0.04824349877870011
	test error:  0.04816388872824073


,train_error,test_error
,Float64,Float64
1,0.0481591,0.0482537
2,0.0482435,0.0481639


In [39]:
hingek4 = convert(DataFrame, X04) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
CSV.write("/Users/kathy908000/github/health-care-research/data/hingek4.csv",  hingek4)
CSV.read("/Users/kathy908000/github/health-care-research/data/hingek4.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
#k=3

In [4]:
loss1 = fill(HuberLoss(),2)
loss2 = fill(HingeLoss(), 917)
loss3 = fill(OrdinalHingeLoss(), 1)
losses = [loss1; loss3; loss2]
losses

920-element Array{Loss,1}:
 HuberLoss(1.0, RealDomain(), 1.0)
 HuberLoss(1.0, RealDomain(), 1.0)
 OrdinalHingeLoss(1, 10, 1.0, OrdinalDomain(1, 10))
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 ⋮
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDomain(), 1.0)
 WeightedHingeLoss(1.0, BoolDom

In [5]:
k = 3
A= df3
rx = UnitOneSparseConstraint()
ry = ZeroReg()
glrm03 = GLRM(A,losses,rx,ry,k)
X03,Y03,ch03 = fit!(glrm03)

Fitting GLRM
Iteration 10: objective value = 482563.07654725574
Iteration 20: objective value = 461330.2897421785


([0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; 1.0 0.0 … 1.0 1.0], [2.9919505998550506 1.2469042536748358 … -1.0965651640812017 -1.5749765002517004; 3.074003062673055 1.2923864122797606 … -1.3052402261447096 -1.0442307116316778; 2.922126418113742 1.231171241626767 … -1.6986662000030162 -2.4770274108621764], ConvergenceHistory("ProxGradGLRM", [Inf, 8.195791819949925e6, 5.956078499382302e6, 4.09231203441685e6, 2.6373012411984527e6, 1.6296923590188506e6, 1.0317402669532349e6, 716974.3799924289, 568354.8996308036, 506051.28120063554  …  460918.8549846397, 460638.88095666555, 460424.4809877514, 460234.35882961215, 460043.5937028405, 459826.0279343162, 459693.47326735355, 459535.9454037331, 459416.3547054248, 459332.00595671055], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 16.01642417907715, 65.3450939655304, 80.592411

In [6]:
Y03

3×920 Array{Float64,2}:
 2.99195  1.2469   7.04899  -1.00119   …  -1.17109  -1.09657  -1.57498
 3.074    1.29239  7.0002   -0.999838     -1.21881  -1.30524  -1.04423
 2.92213  1.23117  7.00079  -1.02169      -1.17105  -1.69867  -2.47703

In [8]:
imputey3 = impute(losses, Y03) 
imputey3d  = convert(DataFrame, imputey3) 
CSV.write("/Users/kathy908000/github/health-care-research/data/iy3.csv",  imputey3d)
CSV.read("/Users/kathy908000/github/health-care-research/data/iy3.csv")

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
,Float64,Float64,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,2.99195,1.2469,7,0,1,0,0,0,0,1,0,0,0
2,3.074,1.29239,7,0,1,0,0,0,0,1,0,0,0
3,2.92213,1.23117,7,0,1,0,0,0,0,1,0,0,0


In [9]:
hingek3= convert(DataFrame, X03) 

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
CSV.write("/Users/kathy908000/github/health-care-research/data/hingek3_2.csv",  hingek3)
CSV.read("/Users/kathy908000/github/health-care-research/data/hingek3_2.csv")


,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# NNMF

In [8]:
# minimize_{X>=0, Y>=0} ||A - XY||^2

k = 10
A = df3
loss = QuadLoss()
r = NonNegConstraint()
glrm = GLRM(A,loss,r,r,k)
X,Y,ch = fit!(glrm)
println("Convergence history:",ch.objective)
    

Fitting GLRM
Iteration 10: objective value = 764537.0790768476
Iteration 20: objective value = 189334.73924219978
Iteration 30: objective value = 161362.85257062083
Iteration 40: objective value = 157023.18324285527
Iteration 50: objective value = 151069.50439275693
Iteration 60: objective value = 145188.07084500446
Iteration 70: objective value = 141302.76335271215
Iteration 80: objective value = 139114.99790250554
Iteration 90: objective value = 137963.30744522018
Convergence history:[Inf, 8.130553586995648e7, 2.398761303180184e6, 2.1148184233406405e6, 2.0232038206393218e6, 1.8252981416761943e6, 1.6932783623200082e6, 1.437978115467547e6, 1.2151400185981703e6, 956005.9864030329, 764537.0790768476, 620886.7172769974, 530063.5752974483, 466363.65305628296, 412671.1453068042, 359283.87048848683, 307650.7349968916, 262058.0281492326, 227402.07294989645, 203927.00288718744, 189334.73924219978, 180378.50478425293, 174673.72917126463, 170740.8323234377, 167997.60647421083, 166046.3546136129,

In [9]:
X

10×10310 Array{Float64,2}:
 0.00746376  0.0158258    0.0         …  0.0819388   0.0478082    0.0
 0.0159935   0.00320387   0.00810761     0.00432487  0.0          0.0
 0.0         0.0221281    0.0            0.00324694  0.0          0.0117319
 0.0         0.0          0.0            0.0125264   0.00883234   0.0
 0.0         0.0          0.0            0.0         0.000113913  0.0
 0.0906721   0.139666     0.082231    …  0.0913632   0.105025     0.0234399
 0.0         0.000331621  0.0            0.0         0.0          0.0
 0.119204    0.012094     0.0331585      0.0290805   0.0276679    0.0311856
 0.128364    0.184366     0.295133       0.148516    0.317999     0.270158
 0.0         0.0          0.0            0.0259988   0.0178199    0.0172448

In [10]:
println("Computing cross validation error for each fold")
params = Params(1.0, max_iter=100, abs_tol=0.0, min_stepsize=.001)
#max_iter=70
train_error, test_error, train_glrms, test_glrms = cross_validate(glrm, nfolds=5, params=params)
df = DataFrame(train_error = train_error, test_error = test_error)

Computing cross validation error for each fold
flattening observations
computing CV folds

forming train and test GLRM for fold 1
training model on 7587682 samples and testing on 1897518
fitting train GLRM for fold 1
Fitting GLRM
Iteration 10: objective value = 108646.87673280195
Iteration 20: objective value = 108233.82956817739
Iteration 30: objective value = 107878.62086500217
Iteration 40: objective value = 107478.83032595043
Iteration 50: objective value = 106965.49226451431
Iteration 60: objective value = 106329.57067913972
Iteration 70: objective value = 105681.31173824564
Iteration 80: objective value = 105125.48956562344
Iteration 90: objective value = 104489.58431235782
Iteration 100: objective value = 103539.33470941114
computing train and test error for fold 1:
	train error: 0.01364571350109242
	test error:  0.03605815738829553

forming train and test GLRM for fold 2
training model on 7587945 samples and testing on 1897255
fitting train GLRM for fold 2
Fitting GLRM
Iteratio

,train_error,test_error
,Float64,Float64
1,0.0136457,0.0360582
2,0.0136396,0.0378168
3,0.0136428,0.0338693
4,0.0136,0.0367899
5,0.0136301,0.0585217
